In [1]:
import json
from typing import List

import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
with open("sentences.json") as f:
    sentenses = json.load(f)

with open("questions.json") as f:
    questions = json.load(f)

In [3]:
quantizer = faiss.IndexFlatIP(768)
index = faiss.IndexIDMap(quantizer)

In [4]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

def encode(sentense: str) -> torch.Tensor:
    tokens = tokenizer(sentense, return_tensors="pt")
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
index.add_with_ids(np.array([encode(s).numpy() for s in sentenses]), np.array(range(0, len(sentenses))))

In [6]:
def search(query: str) -> List[str]:
    encoded_query = encode(query).unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, 5)
    return [sentenses[k] for k in top_k[1][0]]

In [11]:
for q in questions:
    print(f"Question:")
    print(q)
    print("Answers:")
    print("  \n".join(search(q)))
    print("-----")

Question:
How many people have died during Black Death?
Answers:
As of 2018, approximately 37.9 million people are infected with HIV globally.  
A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.  
The death toll of Spanish Flu is estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.  
The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.  
The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.
-----
Question:
Which diseases can be transmitted by animals?
Answers:
A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance mul